<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/assignment7_wordembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload

In [2]:
import os 
os.environ['OMP_NUM_THREADS'] = '64'

In [3]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install --upgrade gensim -qq
    !pip install -U spacy -qq


    basepath = '/content/drive/MyDrive/NLP/assignment7'
    sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import random
import spacy
import textwrap
import joblib
from pathlib import Path
import numpy as np
import gensim
import gensim.downloader as loader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import itertools

In [5]:
import os
os.cpu_count()

4

In [6]:
print(f'spacy-version: {spacy.__version__}  || gensim-version{ gensim.__version__}')

spacy-version: 3.5.2  || gensim-version4.3.1


In [7]:
#!python3 --version

In [8]:
if gensim.models.doc2vec.FAST_VERSION > -1:
    print("Fast gensim version is being used.")
else:
    print("Slow gensim version is being used.")

Fast gensim version is being used.


In [9]:
folder = Path(basepath)/'WordEmbeddings'

In [10]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP/custom-functions']

In [11]:
stackExchange_cleaned_text_file = folder / 'df_raw_small_hw.joblib'    #Folder path was set for accessing the file

In [12]:
#Load the file 
stackExchange_cleaned_text = joblib.load(stackExchange_cleaned_text_file)

In [13]:
type(stackExchange_cleaned_text)

pandas.core.frame.DataFrame

In [14]:
stackExchange_cleaned_text.shape

(438813, 4)

In [15]:
stackExchange_cleaned_text.columns

Index(['Unnamed: 0', 'Id', 'Title', 'Body'], dtype='object')

In [16]:
stackExchange_cleaned_text = stackExchange_cleaned_text.drop(columns=['Unnamed: 0'])

In [17]:
stackExchange_cleaned_text.head(5)

,Id,Title,Body
3595496,7254900,Identify un-activated servers,<p>I've been on a bit of a roll this week and ...
1929804,4905251,How to abort the loading of an external HTML r...,"<p>I'm writing a JavaScript widget, which is i..."
2513704,6173108,Xcode: how do you make it to where clicking on...,<p>I know how to make a button in Xcode with I...
3794169,7453573,Common WQL Monitoring Queries,<p>What WQL queries would you use for monitori...
4341677,8001081,Loading Native Managed and C++ DLL within IIS ...,<p>I have WCF service developed in C# for with...


In [18]:
stackExchange_sentences = [text.split() for text in stackExchange_cleaned_text['Body']]
#len(stackExchange_cleaned_text)
#print(stackExchange_cleaned_text)
#stackExchange_preprocess_sent = []
#for text in stackExchange_cleaned_text['Body']:
  #sentence = preprocess(text)
  #stackExchange_preprocess_sent.append(sentence)

In [19]:
type(stackExchange_sentences)

list

In [20]:
len(stackExchange_sentences)

438813

In [21]:
print(stackExchange_sentences[0])

["<p>I've", 'been', 'on', 'a', 'bit', 'of', 'a', 'roll', 'this', 'week', 'and', 'have', 'deployed', 'around', '30', 'fresh', 'installs', 'of', 'Windows', 'Server', '2008', 'R2', 'using', 'our', 'Datacentre', 'license,', 'inside', 'a', 'VMWare', 'environment.</p>', '<p>However,', 'what', "I've", 'gone', 'and', 'forgotten', 'as', 'part', 'of', 'the', 'setup', 'routine', 'is', 'to', 'activate', 'the', 'servers.</p>', '<p>The', 'last', 'thing', 'I', 'want', 'is', 'next', 'month', 'to', 'find', 'out', 'that', 'our', 'clients', "can't", 'log', 'into', 'our', 'terminal', 'servers', 'because', "I've", 'forgotten', 'to', 'activate', 'them,', 'or', 'for', 'them', 'to', 'see', '"You', 'have', 'been', 'a', 'victim', 'of', 'software', 'piracy"', 'pop', 'up', 'on', 'their', 'desktop.</p>', '<p>Is', 'there', 'any', 'script/dirty', 'hacks', 'that', 'I', 'can', 'use', 'to', 'quickly', 'scan', 'the', 'network', 'for', 'un-activated', 'machines?', "I'm", 'guessing', 'it', 'will', 'probably', 'be', 'a', '

In [22]:
model_stackExchange_CBOW = Word2Vec(sentences=stackExchange_sentences,epochs=10, vector_size=150, window=10, min_count=5, workers = 32, sg=0)
#model_stackExchange_CBOW.build_vocab(sentences=stackExchange_sentences)

#model_stackExchange_CBOW.train(sentences=stackExchange_sentences, total_examples=len(stackExchange_sentences), epochs=10)


In [23]:
type(model_stackExchange_CBOW)

gensim.models.word2vec.Word2Vec

In [25]:
model_stackExchange_CBOW.vector_size

150

In [26]:
model_stackExchange_CBOW.wv.vectors.shape  #There are 356393 unique words in the corpus and each word is represented by 150 dimensional vector

(356393, 150)

In [27]:
model_stackExchange_CBOW.wv.save(str(folder/'model_stackExchange_CBOW.bin'))